In [1]:
import os
import re
import numpy as np
import pandas as pd
from tifffile import imread
from tqdm import tqdm

In [2]:

def assign_nuclei_to_cells_optimized(seg_membrane, seg_nuclei):
    # Find unique membrane labels (excluding background, i.e., label 0)
    membrane_labels = np.unique(seg_membrane)
    membrane_labels = membrane_labels[membrane_labels != 0]  # Exclude background

    # Prepare an empty list to store the results (rows of the table)
    results = []

    # Loop over each membrane label
    for membrane_label in tqdm(membrane_labels):
        # Create a mask for the current membrane label
        membrane_mask = (seg_membrane == membrane_label)
        
        # Get the nucleus labels within the membrane mask
        overlapping_nuclei = seg_nuclei[membrane_mask]
        
        # Use bincount to count occurrences of each label (ignoring background label 0)
        counts = np.bincount(overlapping_nuclei)
        counts[0] = 0  # Set the count of background (label 0) to 0

        # If there are non-zero counts, find the label with the maximum count (i.e., max overlap)
        if counts.max() > 0:
            best_nucleus = np.argmax(counts)  # Label with maximum overlap
            best_overlap = counts[best_nucleus]  # Number of overlapping voxels
            
            # Append the result (membrane label, best nucleus label, overlap count)
            results.append([membrane_label, best_nucleus, best_overlap])
    
    # Convert the results to a DataFrame for easy viewing
    overlap_table = pd.DataFrame(results, columns=["mem_label", "nuc_label", "Overlap Voxels"])
    
    return overlap_table

In [ ]:
# Function to process a folder of TIFF files
def process_tiff_folder(membrane_folder, nuclei_folder):
    all_results = []
    
    for t in range(1, 306):

        print(f'Processing timepoint {t}')
        mem_file = f'final_relabelled_Merged-{t}_cp_masks.tif'
        nuc_file = f'Merged-{t}.tif'

        # Read the membrane and nuclei TIFF files
        membrane = imread(os.path.join(membrane_folder, mem_file))
        nuclei = imread(os.path.join(nuclei_folder, nuc_file))

        # Assign nuclei to cells and get the overlap table for this time point
        overlap_table = assign_nuclei_to_cells_optimized(membrane, nuclei)

        # Remove duplicates by keeping the row with the maximum "Overlap Voxels" for each "Nucleus Label"
        cleaned_overlap_table = overlap_table.loc[
            overlap_table.groupby('nuc_label')['Overlap Voxels'].idxmax()].reset_index(drop=True)

        # Add the 't' column for the current time point
        cleaned_overlap_table['t'] = t

        # Append to the overall results
        all_results.append(cleaned_overlap_table)

    # Concatenate all DataFrames into a single DataFrame
    final_df = pd.concat(all_results, ignore_index=True)
    
    return final_df


membrane_folder = 'split_nuclei_membrane_raw/split_cellpose_results_relabelled/'
nuclei_folder = 'split_nuclei_membrane_raw/VollSeg/StarDist/'
final_df = process_tiff_folder(membrane_folder, nuclei_folder)


Processing timepoint 1


100%|██████████| 643/643 [00:42<00:00, 15.17it/s]


Processing timepoint 2


100%|██████████| 629/629 [00:41<00:00, 15.28it/s]


Processing timepoint 3


...

100%|██████████| 3375/3375 [03:03<00:00, 18.35it/s]


Processing timepoint 304


100%|██████████| 3357/3357 [03:02<00:00, 18.42it/s]


Processing timepoint 305


100%|██████████| 3377/3377 [03:06<00:00, 18.12it/s]


In [ ]:
# Save the final DataFrame to a CSV
final_df.to_csv('./mem_nuc_overlap_table.csv', index=False)

In [7]:
import gc
gc.collect()

0